In [1]:
import pandas as pd
import requests
from os import environ

In [2]:
API_KEY = environ["LAST_FM_API_KEY"]

### What to do
* Clean the Play Activity.csv
* Load in Apple Music Library Tracks.json
* Merge on song and album names

### Load Play Activity.csv

In [2]:
df = pd.read_csv("./data/apple/Apple Music Play Activity.csv", low_memory=False)

### Clean the data

In [3]:
df.dropna(subset=['Album Name'], inplace=True)

In [4]:
df.dropna(subset=['Song Name'], inplace=True)

In [5]:
df = df[df['Media Type'] != 'VIDEO']

In [6]:
df = df[df['Play Duration Milliseconds'] >= 0]

In [7]:
df = df[df['Event Type'] != 'LYRIC_DISPLAY']

In [8]:
df.replace({"Event Start Timestamp": ""}, pd.NA, inplace=True)

In [9]:
df.dropna(subset=["Event Start Timestamp"], inplace=True)

In [10]:
df.describe()

,Apple ID Number,Bundle Version,Camera Option,Carrier Name,Container Artist Name,Container iTunes Playlist ID,Container Personalized ID,Container Season ID,Contingency,Display Count,...,Source Radio Name,Start Position In Milliseconds,Subscribed State,Subscription Bundle ID,Subscription Discovery Mode,Subscription Offer ID,Subscription User ID,UTC Offset In Seconds,Vocal Attenuation Duration,Vocal Attenuation Model ID
count,12244.0,9.173000e+03,0.0,0.0,0.0,5.890000e+02,0.0,0.0,0.0,0.0,...,0.0,1.224400e+04,0.0,0.0,0.0,0.0,8.332000e+03,12244.000000,7291.000000,0.0
mean,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560020e+08,NaN,NaN,NaN,NaN,...,NaN,4.022412e+04,NaN,NaN,NaN,NaN,1.608092e+09,1952.009147,0.867919,NaN
std,0.0,4.441134e-16,NaN,NaN,NaN,9.302435e+02,NaN,NaN,NaN,NaN,...,NaN,2.017859e+05,NaN,NaN,NaN,NaN,0.000000e+00,1794.233291,74.109336,NaN
min,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560003e+08,NaN,NaN,NaN,NaN,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,0.000000,0.000000,NaN
25%,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560010e+08,NaN,NaN,NaN,NaN,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,0.000000,0.000000,NaN
50%,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560026e+08,NaN,NaN,NaN,NaN,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,3600.000000,0.000000,NaN
75%,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560028e+08,NaN,NaN,NaN,NaN,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,3600.000000,0.000000,NaN
max,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560033e+08,NaN,NaN,NaN,NaN,...,NaN,3.820895e+06,NaN,NaN,NaN,NaN,1.608092e+09,7200.000000,6328.000000,NaN


### Load Apple Music Library Tracks.json

In [77]:
library_tracks = pd.read_json('./data/apple/Apple Music Library Tracks.json')

In [78]:
library_tracks['Artist'] = library_tracks['Artist'].str.lower()

In [79]:
# get the genre of each artist
artist_groups = library_tracks.groupby('Artist')

In [104]:
list(artist_groups.get_group("Swingin' Utters".lower())["Genre"].unique())  # raises a KeyError

['Rock', 'Alternative', 'Punk']

In [110]:
def get_genre(artist: str) -> list:
    try:
        genres = list(artist_groups.get_group(artist.lower())["Genre"].unique())
    except (AttributeError, KeyError):
        genres = [None]
    return genres

### Load Apple Music - Play History Daily Tracks.csv

In [21]:
tracks_df = pd.read_csv("./data/apple/Apple Music - Play History Daily Tracks.csv")
tracks_df.head(2)

,Country,Track Identifier,Media type,Date Played,Hours,Play Duration Milliseconds,End Reason Type,Source Type,Play Count,Skip Count,Ignore For Recommendations,Track Reference,Track Description
0,United Kingdom,1116868327,AUDIO,20160828,21,231000,NOT_APPLICABLE,IPHONE,1,0,NaN,NaN,Glass Animals - Youth
1,United Kingdom,1129287609,AUDIO,20160828,21,221000,NOT_APPLICABLE,IPHONE,1,0,NaN,NaN,AlunaGeorge - Mean What I Mean (feat. Leikeli4...


In [22]:
tracks_df.dropna(subset=['Track Description'], inplace=True)

In [23]:
# Split 'Track Description' into 'Artist' and 'Song Name'
split_columns = tracks_df['Track Description'].str.split(' - ', expand=True)
tracks_df['Artist'] = split_columns[0]
tracks_df['Song Name'] = split_columns[1]

In [24]:
tracks_df.head(2)

,Country,Track Identifier,Media type,Date Played,Hours,Play Duration Milliseconds,End Reason Type,Source Type,Play Count,Skip Count,Ignore For Recommendations,Track Reference,Track Description,Artist,Song Name
0,United Kingdom,1116868327,AUDIO,20160828,21,231000,NOT_APPLICABLE,IPHONE,1,0,NaN,NaN,Glass Animals - Youth,Glass Animals,Youth
1,United Kingdom,1129287609,AUDIO,20160828,21,221000,NOT_APPLICABLE,IPHONE,1,0,NaN,NaN,AlunaGeorge - Mean What I Mean (feat. Leikeli4...,AlunaGeorge,Mean What I Mean (feat. Leikeli47 & Dreezy)


In [25]:
filtered_tracks_df = tracks_df.filter(['Artist', 'Song Name'])
filtered_tracks_df = filtered_tracks_df.drop_duplicates()
filtered_tracks_df = filtered_tracks_df.dropna(how='any')

In [26]:
filtered_tracks_df.describe()

,Artist,Song Name
count,5223,5223
unique,2182,5070
top,NOFX,Butterfly
freq,153,4


In [27]:
filtered_tracks_df.head()

,Artist,Song Name
0,Glass Animals,Youth
1,AlunaGeorge,Mean What I Mean (feat. Leikeli47 & Dreezy)
4,Christine and the Queens,Tilted
6,Whitney Houston,I Wanna Dance With Somebody (2000 Remaster)
7,Backstreet Boys,Everybody (Backstreet's Back) [Extended Version]


In [4]:
def get_album(artist: str, song: str) -> str:
    headers = {
        'user-agent': "MyOwnPlaylistRecommender"
    }

    payload = {
        'api_key': API_KEY,
        'method': 'track.getInfo',
        'format': 'json',
        'artist': artist,
        'track': song
    }

    r = requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=payload)
    r.status_code
    try:
        album_name = r.json()  #['track']['album']['title']
    except KeyError:
        album_name = None
    return album_name

In [5]:
def get_album_info(artist: str, album: str) -> dict:
    headers = {
        'user-agent': "MyOwnPlaylistRecommender"
    }

    payload = {
        'api_key': API_KEY,
        'method': 'album.getinfo',
        'format': 'json',
        'artist': artist,
        'album': album
    }

    r = requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=payload)
    try:
        album_details = r.json()  #['track']['album']['title']
    except KeyError:
        album_details = None
    return album_details

In [6]:
def get_artist_info(artist: str) -> dict:
    headers = {
        'user-agent': "MyOwnPlaylistRecommender"
    }

    payload = {
        'api_key': API_KEY,
        'method': 'artist.gettopalbums',
        'format': 'json',
        'artist': artist
    }

    r = requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=payload)
    try:
        artist_details = r.json()  #['track']['album']['title']
    except KeyError:
        artist_details = None
    return artist_details

In [7]:
artist_info = get_artist_info("Alicia Keys")

In [8]:
# artist_info["topalbums"]
[album["name"] for album in artist_info["topalbums"]["album"]]

['The Diary of Alicia Keys',
 'Songs in A Minor',
 'The Element of Freedom',
 'As I Am',
 'Girl on Fire',
 'As I Am (Expanded Edition)',
 'Girl On Fire (Remixes) - EP',
 'No One (Remixes)',
 'Unplugged',
 'Alicia',
 'Songs In A Minor (Expanded Edition)',
 'As I Am - The Super Edition',
 "Un-Thinkable (I'm Ready) (Feat. Drake) [Remix]",
 "It's On Again (feat. Kendrick Lamar) [From The Amazing Spider-Man 2 Soundtrack]",
 'HERE',
 'The Life (Sampler)',
 'Underdog',
 'Songs in A Minor (10th Anniversary Edition) (Deluxe Edition)',
 'In Common',
 'KEYS II',
 'So Done (feat. Khalid)',
 'Keys',
 'Vault Playlist Vol. 1',
 'Spotify Singles',
 'Diary of Alicia Keys',
 'The Element Of Freedom (Deluxe)',
 '(null)',
 'Vault Volume 1',
 'Queen Charlotte: A Bridgerton Story (Covers from the Netflix Series)',
 'Show Me Love',
 "Doesn't Mean Anything",
 "If I Ain't Got You EP",
 'We Are Here',
 'Love Looks Better',
 'Show Me Love (feat. 21 Savage & Miguel)',
 "If I Ain't Got You",
 'No One',
 'Time Mach

In [9]:
import re
x = 'Carolina ("Where The Crawdads Sing"'
x = re.sub("\(.*?\)|\[.*?\]","",x)
print(x)

Carolina ("Where The Crawdads Sing"


In [10]:
the_album = get_album("Alicia Keys", "Plentiful (Originals) [feat. Pusha T]")
print(the_album)

{'track': {'name': 'Plentiful (Originals) [feat. Pusha T]', 'url': 'https://www.last.fm/music/Alicia+Keys/_/Plentiful+(Originals)+%5Bfeat.+Pusha+T%5D', 'duration': '188000', 'streamable': {'#text': '0', 'fulltrack': '0'}, 'listeners': '1013', 'playcount': '3758', 'artist': {'name': 'Alicia Keys', 'mbid': '8ef1df30-ae4f-4dbd-9351-1a32b208a01e', 'url': 'https://www.last.fm/music/Alicia+Keys'}, 'toptags': {'tag': []}}}


In [11]:
proper_album = get_album_info("Alicia Keys", "Keys II")

In [12]:
# proper_album["album"]["tracks"]
[track["name"] for track in proper_album["album"]["tracks"]["track"]]

['Plentiful (Originals) (feat. Pusha T)',
 'Skydive (Originals)',
 'Best Of Me (Originals)',
 'Dead End Road (Originals)',
 'Is It Insane (Originals)',
 'Billions (Originals)',
 'Love When You Call My Name (Originals)',
 'Only You (Originals)',
 'Daffodils (Originals)',
 'Old Memories (Originals)',
 'Nat King Cole (Originals)',
 'Paper Flowers (Originals) (feat. Brandi Carlile)',
 'Like Water (Originals)',
 'Stay (feat. Lucky Daye)',
 'In Common - Black Coffee Remix',
 'Keys',
 'Only You (Unlocked)',
 'Skydive (Unlocked)',
 'Best Of Me (Unlocked)',
 'LALA (Unlocked) (feat. Swae Lee)',
 'Nat King Cole (Unlocked) (feat. Lil Wayne)',
 'Is It Insane (Unlocked)',
 'Come For Me (Unlocked) (feat. Khalid & Lucky Daye)',
 'Old Memories (Unlocked)',
 'Dead End Road (Unlocked)',
 'Love When You Call My Name (Unlocked)',
 'Daffodils (Unlocked)',
 'Billions (Unlocked)',
 'Trillions (feat. Brent Faiyaz)',
 'In Common - Kaskade Radio Mix']

In [13]:
the_album

{'track': {'name': 'Plentiful (Originals) [feat. Pusha T]',
  'url': 'https://www.last.fm/music/Alicia+Keys/_/Plentiful+(Originals)+%5Bfeat.+Pusha+T%5D',
  'duration': '188000',
  'streamable': {'#text': '0', 'fulltrack': '0'},
  'listeners': '1013',
  'playcount': '3758',
  'artist': {'name': 'Alicia Keys',
   'mbid': '8ef1df30-ae4f-4dbd-9351-1a32b208a01e',
   'url': 'https://www.last.fm/music/Alicia+Keys'},
  'toptags': {'tag': []}}}

In [30]:
filtered_tracks_df['Album'] = filtered_tracks_df.apply(lambda x: get_album(x['Artist'], x['Song Name']), axis=1)

In [31]:
filtered_tracks_df.to_csv("filtered_tracks.csv")

In [88]:
filtered_library_df = library_tracks.filter(['Artist', 'Title', 'Album'])
filtered_library_df = filtered_library_df.drop_duplicates()
filtered_library_df.head()

,Artist,Title,Album
0,shy fx & t-power,Shake Ur Body (feat. Di),Set It Off
1,the cranberries,Zombie,No Need To Argue (Remastered 2020)
2,spooks,Karma Hotel,"S.I.O.S.O.S., Vol. One (Remastered)"
3,major lazer,Cold Water (feat. Justin Bieber & MØ),Cold Water (feat. Justin Bieber & MØ) - Single
4,taylor swift,...Ready For It?,reputation


In [89]:
filtered_library_df.describe()

,Artist,Title,Album
count,7988,8022,7821
unique,1453,7525,976
top,bbc radio 1,Intro,<Unknown>
freq,411,18,267


# Function to find info

In [90]:
from collections import defaultdict

In [ ]:
artist_dict = defaultdict(list)

def find_artist_in_library(album: str, song: str):
    artist = None
    artists_from_track = list(filtered_tracks_df[filtered_tracks_df['Song Name'] == song]["Artist"].unique())
    artists_from_album = list(filtered_library_df[filtered_library_df['Album'] == album]["Artist"].unique())
    artists_from_non_exact_song_match = list(filtered_tracks_df[filtered_tracks_df.map(lambda x: song.lower() in x.lower() if isinstance(x, str) else False)["Song Name"]]["Artist"].unique())
    potential_artist = list(set(artists_from_track).intersection(artists_from_album))
    if len(artists_from_track) == 1:
        artist_dict[artists_from_track[0]].append(album)
        artist = artists_from_track[0]
    elif len(potential_artist) == 1:
        artist = potential_artist[0]
    elif not artist:
        for a in artists_from_track:
            try:
                tmp_album_list = artist_dict[a]
                if album in tmp_album_list:
                    artist = a
                    break
            except KeyError:
                pass
    elif len(artists_from_non_exact_song_match) == 1:
            artist = artists_from_non_exact_song_match[0]
    else:
        artist = "I CANNOT FIND THE ARTIST!!!"
        # headers = {"user-agent": "MyOwnPlaylistRecommender"}
        # payload = {
        #     "api_key": API_KEY,
        #     "method": "artist.getInfo",
        #     "format": "json",
        # }
        # response = requests.get(f'http://ws.audioscrobbler.com/2.0/?method=track.getInfo&track={song}&artist={artist}&api_key={API_KEY}&format=json')
        # data = response.json()
    return artist

def find_artist_from_track_only(song: str):
    return filtered_tracks_df[filtered_tracks_df['Song Name'] == song]["Artist"].to_list()

def find_artist_from_album_only(album: str):
    return list(filtered_library_df[filtered_library_df['Album'] == album]["Artist"].unique())

In [92]:
def non_exact_track_names(song: str):
    artist = pd.NA
    list_of_artists = filtered_tracks_df[filtered_tracks_df.map(lambda x: song.lower() in x.lower() if isinstance(x, str) else False)["Song Name"]]["Artist"].to_list()
    if list_of_artists:
        artist = list_of_artists[0]
    return artist

In [93]:
df['Artist from track'] = df.apply(lambda x: find_artist_from_track_only(x["Song Name"]), axis=1)
df['Artist from non exact matches'] = df.apply(lambda x: non_exact_track_names(x["Song Name"]), axis=1)
df['Artist'] = df.apply(lambda x: find_artist_in_library(x["Album Name"], x["Song Name"]), axis=1)

In [94]:
small_df = df[['Song Name', 'Album Name', 'Artist from track', 'Artist from non exact matches', 'Artist']]

In [95]:
small_df.describe()

,Song Name,Album Name,Artist from track,Artist from non exact matches,Artist
count,12244,12244,12244,11796,11004
unique,5076,2868,2043,1994,1912
top,The Marriage of Figaro : Act IV,"Mozart: Don Giovanni, K. 527",[],The Offspring,The Offspring
freq,55,312,900,676,666


In [96]:
small_df.head(5)

,Song Name,Album Name,Artist from track,Artist from non exact matches,Artist
0,Science Fiction Double Feature,Are a Drag,[Me First and The Gimme Gimmes],Me First and The Gimme Gimmes,Me First and The Gimme Gimmes
1,Don't Stop Believin' (2022 Remaster),Escape (2022 Remaster),[Journey],Journey,Journey
2,Watch Me as I Fall,Straight Ahead,[Pennywise],Pennywise,Pennywise
4,Know It All,"Acoustic, Vol. 2",[Joey Cape],Joey Cape,Joey Cape
5,Alison's Disease,"Acoustic, Vol. 2","[Joey Cape, Lagwagon]",Joey Cape,Joey Cape


In [97]:
small_df["Final Artist"] = small_df["Artist"].fillna(small_df["Artist from non exact matches"])

C:\Users\ftayl\AppData\Local\Temp\ipykernel_15056\1836027413.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_df["Final Artist"] = small_df["Artist"].fillna(small_df["Artist from non exact matches"])


In [98]:
no_artists = small_df["Final Artist"] == "I CANNOT FIND THE ARTIST!!!"

In [99]:
small_df[no_artists]

,Song Name,Album Name,Artist from track,Artist from non exact matches,Artist,Final Artist


In [111]:
small_df['Genre'] = small_df["Final Artist"].apply(get_genre)

C:\Users\ftayl\AppData\Local\Temp\ipykernel_15056\3057049876.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_df['Genre'] = small_df["Final Artist"].apply(get_genre)


In [112]:
small_df.head(50)

,Song Name,Album Name,Artist from track,Artist from non exact matches,Artist,Final Artist,Genre
0,Science Fiction Double Feature,Are a Drag,[Me First and The Gimme Gimmes],Me First and The Gimme Gimmes,Me First and The Gimme Gimmes,Me First and The Gimme Gimmes,"[Punk, Rock, Punk Rock, Alternative Rock, Othe..."
1,Don't Stop Believin' (2022 Remaster),Escape (2022 Remaster),[Journey],Journey,Journey,Journey,[Classic Rock]
2,Watch Me as I Fall,Straight Ahead,[Pennywise],Pennywise,Pennywise,Pennywise,"[Other, Rock, Alternative]"
4,Know It All,"Acoustic, Vol. 2",[Joey Cape],Joey Cape,Joey Cape,Joey Cape,"[Rock, Punk]"
5,Alison's Disease,"Acoustic, Vol. 2","[Joey Cape, Lagwagon]",Joey Cape,Joey Cape,Joey Cape,"[Rock, Punk]"
6,Heart Of A Woman,Heart Of A Woman - Single,[Summer Walker],Summer Walker,Summer Walker,Summer Walker,[None]
7,Elfentanz,Price: Elfentanz - Single,[Randall Goosby & Zhu Wang],Randall Goosby & Zhu Wang,Randall Goosby & Zhu Wang,Randall Goosby & Zhu Wang,[None]
8,"Le Carnaval des animaux, R. 125: XIII. Les Cyg...",Golden Oldies – More Favourite Encores,[Laura van der Heijden & Brodsky Quartet],Laura van der Heijden & Brodsky Quartet,Laura van der Heijden & Brodsky Quartet,Laura van der Heijden & Brodsky Quartet,[None]
12,Inside Out,Camila,[Camila Cabello],Camila Cabello,Camila Cabello,Camila Cabello,[Pop]
21,Relax,Bang! (Here Comes A Supernova),[Frankie Goes to Hollywood],A Tribe Called Quest,Frankie Goes to Hollywood,Frankie Goes to Hollywood,[Vocal]


### Merge dataframes

In [70]:
library_rename = {"Title": "Song Name", "Album": "Album Name"}
library_tracks.rename(columns=library_rename, inplace=True)
library_tracks.head()

,Content Type,Track Identifier,Song Name,Sort Name,Artist,Sort Artist,Composer,Is Part of Compilation,Album Name,Sort Album,...,Tag Matched Track Identifier,Comments,Rating,Album Rating,Remember Playback Position,Beats Per Minute,Date of Last Skip,Track Like Rating,Favorite Status - Track,Favorite Date - Track
0,Song,182997530,Shake Ur Body (feat. Di),Shake Ur Body (feat. Di),Shy FX & T-Power,Shy FX & T-Power,"A. Williams, D Joseph, Shy FX & T-Power, A.Wil...",True,Set It Off,Set It Off,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Song,182997534,Zombie,Zombie,The Cranberries,Cranberries,Dolores O'Riordan,False,No Need To Argue (Remastered 2020),No Need To Argue (Remastered 2020),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Song,182997538,Karma Hotel,Karma Hotel,Spooks,Spooks,NaN,False,"S.I.O.S.O.S., Vol. One (Remastered)","S.I.O.S.O.S., Vol. One (Remastered)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Song,182997542,Cold Water (feat. Justin Bieber & MØ),Cold Water (feat. Justin Bieber & MØ),Major Lazer,Major Lazer,"Diplo, benny blanco, Jr. Blender & King Henry",False,Cold Water (feat. Justin Bieber & MØ) - Single,Cold Water (feat. Justin Bieber & MØ) - Single,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Song,182997546,...Ready For It?,...Ready For It?,Taylor Swift,Taylor Swift,"Taylor Swift, Max Martin, Shellback & Ali Payami",False,reputation,reputation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
library_tracks.describe()

,Track Identifier,Track Year,Track Number On Album,Track Count On Album,Disc Number Of Album,Disc Count Of Album,Track Duration,Track Play Count,Skip Count,Playlist Only Track,Purchased Track Identifier,Apple Music Track Identifier,Display Work Name,Audio Matched Track Identifier,Tag Matched Track Identifier,Rating,Album Rating,Remember Playback Position,Beats Per Minute,Favorite Status - Track
count,8.084000e+03,8084.000000,8084.000000,8084.000000,8084.000000,8084.000000,8.084000e+03,8084.00000,8084.000000,396.000000,4.760000e+02,3.242000e+03,2.0,1.100000e+01,2.981000e+03,7608.0,7608.0,7608.0,62.000000,2.0
mean,1.830714e+08,1829.974518,9.391019,1.423924,1.015339,0.918605,3.371886e+05,0.26472,0.001979,0.994949,1.224386e+09,1.014330e+09,1.0,9.157942e+08,5.100509e+08,0.0,0.0,0.0,595.467742,1.0
std,4.267514e+04,560.869846,14.318227,4.791640,0.194600,0.524359,4.875245e+05,1.01489,0.047148,0.070977,5.051184e+08,5.536713e+08,0.0,9.360458e+00,4.748565e+08,0.0,0.0,0.0,3888.586956,0.0
min,1.829975e+08,0.000000,0.000000,0.000000,1.000000,0.000000,2.856000e+03,0.00000,0.000000,0.000000,7.902380e+05,7.902380e+05,1.0,9.157942e+08,1.519840e+05,0.0,0.0,0.0,82.000000,1.0
25%,1.830344e+08,1997.000000,3.000000,0.000000,1.000000,1.000000,1.714085e+05,0.00000,0.000000,1.000000,8.017567e+08,3.621335e+08,1.0,9.157942e+08,2.073413e+08,0.0,0.0,0.0,89.000000,1.0
50%,1.830715e+08,2002.000000,7.000000,0.000000,1.000000,1.000000,2.217530e+05,0.00000,0.000000,1.000000,1.443129e+09,1.169861e+09,1.0,9.157942e+08,2.888895e+08,0.0,0.0,0.0,95.500000,1.0
75%,1.831084e+08,2004.000000,12.000000,0.000000,1.000000,1.000000,2.716145e+05,0.00000,0.000000,1.000000,1.621611e+09,1.467621e+09,1.0,9.157942e+08,8.368004e+08,0.0,0.0,0.0,104.750000,1.0
max,1.831453e+08,2024.000000,224.000000,61.000000,5.000000,10.000000,7.085688e+06,13.00000,2.000000,1.000000,1.777152e+09,1.778048e+09,1.0,9.157942e+08,1.478458e+09,0.0,0.0,0.0,30720.000000,1.0


In [63]:
# merge play activity df and library tracks directly
# new_merged_df = df.merge(library_tracks, left_on=['Song Name', 'Album Name'], right_on=['Song Name', 'Album Name'])
new_merged_df = pd.merge(df, library_tracks, on=['Song Name', 'Album Name'], how='left')

In [64]:
small_df = new_merged_df[['Song Name', 'Album Name', 'Artist']]

In [72]:
small_df.describe()

,Song Name,Album Name,Artist from track,Artist from non exact matches,Artist,Final Artist
count,12244,12244,12244,11796,11046,11796
unique,5076,2868,2043,1994,1919,2065
top,The Marriage of Figaro : Act IV,"Mozart: Don Giovanni, K. 527",[],The Offspring,The Offspring,The Offspring
freq,55,312,900,676,666,668


In [73]:
small_df.head(50)

,Song Name,Album Name,Artist from track,Artist from non exact matches,Artist,Final Artist
0,Science Fiction Double Feature,Are a Drag,[Me First and The Gimme Gimmes],Me First and The Gimme Gimmes,Me First and The Gimme Gimmes,Me First and The Gimme Gimmes
1,Don't Stop Believin' (2022 Remaster),Escape (2022 Remaster),[Journey],Journey,Journey,Journey
2,Watch Me as I Fall,Straight Ahead,[Pennywise],Pennywise,Pennywise,Pennywise
4,Know It All,"Acoustic, Vol. 2",[Joey Cape],Joey Cape,Joey Cape,Joey Cape
5,Alison's Disease,"Acoustic, Vol. 2","[Joey Cape, Lagwagon]",Joey Cape,Joey Cape,Joey Cape
6,Heart Of A Woman,Heart Of A Woman - Single,[Summer Walker],Summer Walker,Summer Walker,Summer Walker
7,Elfentanz,Price: Elfentanz - Single,[Randall Goosby & Zhu Wang],Randall Goosby & Zhu Wang,Randall Goosby & Zhu Wang,Randall Goosby & Zhu Wang
8,"Le Carnaval des animaux, R. 125: XIII. Les Cyg...",Golden Oldies – More Favourite Encores,[Laura van der Heijden & Brodsky Quartet],Laura van der Heijden & Brodsky Quartet,Laura van der Heijden & Brodsky Quartet,Laura van der Heijden & Brodsky Quartet
12,Inside Out,Camila,[Camila Cabello],Camila Cabello,Camila Cabello,Camila Cabello
21,Relax,Bang! (Here Comes A Supernova),[Frankie Goes to Hollywood],A Tribe Called Quest,Frankie Goes to Hollywood,Frankie Goes to Hollywood


In [74]:
small_df[small_df['Song Name'] == "Science Fiction Double Feature"]

,Song Name,Album Name,Artist from track,Artist from non exact matches,Artist,Final Artist
0,Science Fiction Double Feature,Are a Drag,[Me First and The Gimme Gimmes],Me First and The Gimme Gimmes,Me First and The Gimme Gimmes,Me First and The Gimme Gimmes
238,Science Fiction Double Feature,Are a Drag,[Me First and The Gimme Gimmes],Me First and The Gimme Gimmes,Me First and The Gimme Gimmes,Me First and The Gimme Gimmes
459,Science Fiction Double Feature,Are a Drag,[Me First and The Gimme Gimmes],Me First and The Gimme Gimmes,Me First and The Gimme Gimmes,Me First and The Gimme Gimmes
5643,Science Fiction Double Feature,Are a Drag,[Me First and The Gimme Gimmes],Me First and The Gimme Gimmes,Me First and The Gimme Gimmes,Me First and The Gimme Gimmes
13941,Science Fiction Double Feature,Are a Drag,[Me First and The Gimme Gimmes],Me First and The Gimme Gimmes,Me First and The Gimme Gimmes,Me First and The Gimme Gimmes
19935,Science Fiction Double Feature,Are a Drag,[Me First and The Gimme Gimmes],Me First and The Gimme Gimmes,Me First and The Gimme Gimmes,Me First and The Gimme Gimmes


In [ ]:
another_merged_df = pd.merge(small_df, library_tracks, on=['Song Name', 'Album Name'], how='left')

In [50]:
library_tracks[library_tracks['Song Name'] == "Watch Me as I Fall"][["Artist", "Song Name","Track Play Count", "Track Identifier", "Audio Matched Track Identifier", "Apple Music Track Identifier", "Tag Matched Track Identifier"]]

,Artist,Song Name,Track Play Count,Track Identifier,Audio Matched Track Identifier,Apple Music Track Identifier,Tag Matched Track Identifier
6729,Pennywise,Watch Me as I Fall,1,183120646,NaN,1.485038e+09,271944506.0


#### Convert genre to list

In [35]:
new_merged_df.describe()

,Apple ID Number,Bundle Version,Camera Option,Carrier Name,Container Artist Name,Container iTunes Playlist ID,Container Personalized ID,Container Season ID,Contingency,Display Count,...,Purchased Track Identifier,Apple Music Track Identifier,Display Work Name,Audio Matched Track Identifier,Tag Matched Track Identifier,Rating,Album Rating,Remember Playback Position,Beats Per Minute,Favorite Status - Track
count,12244.0,9.173000e+03,0.0,0.0,0.0,5.890000e+02,0.0,0.0,0.0,0.0,...,4.510000e+02,8.930000e+02,4.0,0.0,4.480000e+02,523.0,523.0,523.0,1.0,1.0
mean,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560020e+08,NaN,NaN,NaN,NaN,...,1.055523e+09,9.382313e+08,1.0,NaN,4.636204e+08,0.0,0.0,0.0,95.0,1.0
std,0.0,4.441134e-16,NaN,NaN,NaN,9.302435e+02,NaN,NaN,NaN,NaN,...,5.073269e+08,5.227850e+08,0.0,NaN,3.810051e+08,0.0,0.0,0.0,NaN,NaN
min,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560003e+08,NaN,NaN,NaN,NaN,...,7.902380e+05,7.902380e+05,1.0,NaN,1.420776e+06,0.0,0.0,0.0,95.0,1.0
25%,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560010e+08,NaN,NaN,NaN,NaN,...,4.674620e+08,3.672050e+08,1.0,NaN,2.145122e+08,0.0,0.0,0.0,95.0,1.0
50%,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560026e+08,NaN,NaN,NaN,NaN,...,1.167031e+09,1.117509e+09,1.0,NaN,2.916631e+08,0.0,0.0,0.0,95.0,1.0
75%,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560028e+08,NaN,NaN,NaN,NaN,...,1.470675e+09,1.440953e+09,1.0,NaN,7.241000e+08,0.0,0.0,0.0,95.0,1.0
max,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560033e+08,NaN,NaN,NaN,NaN,...,1.752032e+09,1.752032e+09,1.0,NaN,1.444221e+09,0.0,0.0,0.0,95.0,1.0


In [63]:
new_merged_df["Genre"]

0          NaN
1          NaN
2        Other
3          NaN
4          NaN
         ...  
12239      NaN
12240      NaN
12241      NaN
12242      NaN
12243      NaN
Name: Genre, Length: 12244, dtype: object

In [64]:
new_merged_df["Genre"] = new_merged_df["Genre"].apply(lambda x: [x])

In [65]:
new_merged_df["Genre"]

0          [nan]
1          [nan]
2        [Other]
3          [nan]
4          [nan]
          ...   
12239      [nan]
12240      [nan]
12241      [nan]
12242      [nan]
12243      [nan]
Name: Genre, Length: 12244, dtype: object

In [66]:
# Display the merged DataFrame
new_merged_df.head()

,Age Bucket,Album Name,Apple ID Number,Apple Music Subscription,Auto Play,Build Version,Bundle Version,Camera Option,Carrier Name,Client Build Version,...,Tag Matched Track Identifier,Comments,Rating,Album Rating,Remember Playback Position,Beats Per Minute,Date of Last Skip,Track Like Rating,Favorite Status - Track,Favorite Date - Track
0,NaN,Are a Drag,277175890,True,AUTO_OFF,"Music/3.1 iOS/18.1 model/iPhone14,4 hwp/t8110 ...",3.1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,45-54,Escape (2022 Remaster),277175890,True,AUTO_ON_CONTENT_UNSUPPORTED,"Music/3.1 iOS/17.2.1 model/iPhone14,4 hwp/t811...",3.1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Straight Ahead,277175890,True,AUTO_OFF,"Music/3.1 iOS/18.0 model/iPhone14,4 hwp/t8110 ...",3.1,NaN,NaN,NaN,...,271944506.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,NaN,"Acoustic, Vol. 2",277175890,True,AUTO_ON_CONTENT_UNSUPPORTED,"Music/3.1 iOS/15.6 model/iPhone12,8 hwp/t8030 ...",3.1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,45-54,"Acoustic, Vol. 2",277175890,True,AUTO_ON,"Music/3.1 iOS/17.2.1 model/iPhone14,4 hwp/t811...",3.1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
new_merged_df.columns

Index(['Age Bucket', 'Album Name', 'Apple ID Number',
       'Apple Music Subscription', 'Auto Play', 'Build Version',
       'Bundle Version', 'Camera Option', 'Carrier Name',
       'Client Build Version',
       ...
       'Tag Matched Track Identifier', 'Comments', 'Rating', 'Album Rating',
       'Remember Playback Position', 'Beats Per Minute', 'Date of Last Skip',
       'Track Like Rating', 'Favorite Status - Track',
       'Favorite Date - Track'],
      dtype='object', length=179)

In [68]:
end_reason_dict = {
    "EXITED_APPLICATION": "logout",
    "FAILED_TO_LOAD": "track_error",
    "MANUALLY_SELECTED_PLAYBACK_OF_A_DIFF_ITEM": "selected_diff_item",
    "NATURAL_END_OF_TRACK": "track_done",
    "NOT_APPLICABLE": "unknown",
    "OTHER": "uknown",
    "PLAYBACK_MANUALLY_PAUSED": "pause",
    "PLAYBACK_SUSPENDED": "suspended",
    "SCRUB_BEGIN": "scrub_begin",
    "SCRUB_END": "scrub_end",
    "TRACK_SKIPPED_BACKWARDS": "back_button",
    "TRACK_SKIPPED_FORWARDS": "forward_button",
    pd.NA: "unknown"
}

In [69]:
shuffle_dict = {
    "SHUFFLE_ON": "On",
    "SHUFFLE_OFF": "Off",
    "SHUFFLE_UNKNOWN": "Unknown"
}

In [70]:
from collections import defaultdict

def constant_factory(value):
    return lambda: value

country_dict = defaultdict(constant_factory("Unknown"))
country_list = [
    ("GB", "United Kingdom"),
    ("AL", "Albania"),
    ("ES", "Spain"),
    ("IE", "Ireland"),
    ("US", "United States")
]
for k, v in country_list:
    country_dict[k] = v

In [71]:
country_dict

defaultdict(<function __main__.constant_factory.<locals>.<lambda>()>,
            {'GB': 'United Kingdom',
             'AL': 'Albania',
             'ES': 'Spain',
             'IE': 'Ireland',
             'US': 'United States'})

In [72]:
new_merged_df.replace({"IP Country Code": pd.NA}, "unknown", inplace=True)

In [73]:
new_merged_df["IP Country Code"]

0             GB
1             GB
2             GB
3        unknown
4             GB
          ...   
12239    unknown
12240         GB
12241    unknown
12242         IE
12243         GB
Name: IP Country Code, Length: 12244, dtype: object

In [74]:
new_merged_df.iloc[2][["Album Name", "Song Name", "IP Country Code"]]

Album Name             Straight Ahead
Song Name          Watch Me as I Fall
IP Country Code                    GB
Name: 2, dtype: object

In [75]:
new_merged_df["Datetime"] = pd.to_datetime(new_merged_df["Event Start Timestamp"], format='mixed')
new_merged_df["Day name"] = new_merged_df["Datetime"].dt.day_name()
new_merged_df["Day number"] = new_merged_df["Datetime"].dt.day
new_merged_df["Month number"] = new_merged_df["Datetime"].dt.month
new_merged_df["Year"] = new_merged_df["Datetime"].dt.year
new_merged_df["Hour"] = new_merged_df["Datetime"].dt.hour
new_merged_df["Song and Artist name"] = new_merged_df["Song Name"] + " | " + new_merged_df["Artist"]
new_merged_df["Platform"] = new_merged_df["Device OS Name"] + " | " + new_merged_df["Device Type"]+ " | " + new_merged_df["Device OS Version"]
new_merged_df["Milliseconds played"] = new_merged_df["Play Duration Milliseconds"]
new_merged_df.replace({"End Reason Type": end_reason_dict}, inplace=True)
new_merged_df.replace({"Shuffle Play": shuffle_dict}, inplace=True)
new_merged_df.replace({"IP Country Code": country_dict}, inplace=True)
new_merged_df["Latitude"] = new_merged_df["IP Latitude"]
new_merged_df["Longitude"] = new_merged_df["IP Longitude"]

In [76]:
new_merged_df.columns

Index(['Age Bucket', 'Album Name', 'Apple ID Number',
       'Apple Music Subscription', 'Auto Play', 'Build Version',
       'Bundle Version', 'Camera Option', 'Carrier Name',
       'Client Build Version',
       ...
       'Day name', 'Day number', 'Month number', 'Year', 'Hour',
       'Song and Artist name', 'Platform', 'Milliseconds played', 'Latitude',
       'Longitude'],
      dtype='object', length=190)

In [77]:
new_merged_df["End Reason Type"].head(10)

0          pause
1     track_done
2     track_done
3     track_done
4    scrub_begin
5     track_done
6     track_done
7     track_done
8          pause
9    scrub_begin
Name: End Reason Type, dtype: object

In [78]:
rename_columns = {
    "Album Name": "Album name",
    "Song Name": "Song name",
    "End Reason Type": "End reason",
    "Shuffle Play": "Shuffle",
    "IP Country Code": "Country"
}

In [79]:
columns_to_keep = [
    "Datetime",
    "Day name",
    "Day number",
    "Month number",
    "Year",
    "Hour",
    "Artist",
    "Album name",
    "Song name",
    "Song and Artist name",
    "Genre",
    "Platform",
    "Milliseconds played",
    "End reason",
    "Shuffle",
    "Country",
    "Latitude",
    "Longitude"
]

In [80]:
apple_df = new_merged_df.rename(columns=rename_columns)[columns_to_keep]
apple_df.head()

,Datetime,Day name,Day number,Month number,Year,Hour,Artist,Album name,Song name,Song and Artist name,Genre,Platform,Milliseconds played,End reason,Shuffle,Country,Latitude,Longitude
0,2024-11-14 15:00:00.836000+00:00,Thursday,14,11,2024,15,NaN,Are a Drag,Science Fiction Double Feature,NaN,[nan],iOS | IPHONE | 18.1,77532.0,pause,Off,United Kingdom,51.50,-0.12
1,2024-01-11 19:51:31.864000+00:00,Thursday,11,1,2024,19,NaN,Escape (2022 Remaster),Don't Stop Believin' (2022 Remaster),NaN,[nan],iOS | IPHONE | 17.2.1,249000.0,track_done,Off,United Kingdom,50.83,-1.07
2,2024-09-26 12:03:26.174000+00:00,Thursday,26,9,2024,12,Pennywise,Straight Ahead,Watch Me as I Fall,Watch Me as I Fall | Pennywise,[Other],iOS | IPHONE | 18.0,130455.0,track_done,Off,United Kingdom,51.50,-0.12
3,2022-08-09 11:17:05.765000+00:00,Tuesday,9,8,2022,11,NaN,"Acoustic, Vol. 2",Know It All,NaN,[nan],iOS | IPHONE | 15.6,202849.0,track_done,Off,unknown,NaN,NaN
4,2024-01-05 10:14:09.839000+00:00,Friday,5,1,2024,10,NaN,"Acoustic, Vol. 2",Alison's Disease,NaN,[nan],iOS | IPHONE | 17.2.1,119500.0,scrub_begin,Off,United Kingdom,50.83,-1.07


In [81]:
apple_df["Country"]

0        United Kingdom
1        United Kingdom
2        United Kingdom
3               unknown
4        United Kingdom
              ...      
12239           unknown
12240    United Kingdom
12241           unknown
12242           Ireland
12243    United Kingdom
Name: Country, Length: 12244, dtype: object